In [137]:
import torch
torch.__version__

'1.12.0+cu116'

In [140]:
a = torch.DoubleTensor([1,2])
torch.Tensor(a)

tensor([1., 2.], dtype=torch.float64)

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, Normalizer
import os

In [32]:
train = pd.read_csv('./train.csv')

In [39]:
train = train.sort_values(['PID', 'UID']).reset_index(drop=True)

add before ef

In [50]:
bef = []
PID = ''
mean_ef = 61
for index in range(len(train)):
    col = train.loc[index, :]
    if PID != col['PID']:
        bef.append(mean_ef)
        PID = col['PID']
        EF = col['EF']
    else:
        bef.append(EF)
        EF = col['EF']

In [51]:
train['BEF'] = bef

In [53]:
train.to_csv('./train.csv', index=False)

count ef under 50 and 35

In [3]:
def transfer_target(df_col):
    if df_col <= 35:
        return 0
    elif 35 < df_col <= 50:
        return 1
    else:
        return 2
# train['target'] = train['EF'].apply(lambda x: 1-(((x-10)*1.25)/100))
train['target'] = train['EF'].apply(transfer_target)

In [ ]:
train['target'].value_counts()

In [5]:
train[train['EF']<50]

,PID,UID,EF,target
1,P02046,U09320,27.5,0
2,P01273,U05782,35.0,0
10,P02355,U10726,22.5,0
17,P02201,U10019,40.0,1
38,P02863,U13030,40.0,1
...,...,...,...,...
9781,P01045,U04733,40.0,1
9786,P01023,U04630,47.5,1
9796,P02090,U09527,37.5,1
9797,P00193,U00857,22.5,0


split data

In [6]:
train.to_csv('./train.csv', index=False)

In [91]:
keep = []
PID = ''
for index in range(len(train)):
    col = train.loc[index, :]
    if PID != col['PID']:
        PID = col['PID']
    else:
        keep.append(index)

In [92]:
train = train.reset_index()
train = train[train['index'].isin(keep)]
train.drop('index', axis=1, inplace=True)

In [93]:
train

,PID,UID,EF,BEF
1,P00001,U00002,70.0,61.0
2,P00001,U00003,68.0,70.0
3,P00001,U00004,69.0,68.0
5,P00002,U00008,62.0,61.0
7,P00003,U00012,55.0,62.0
...,...,...,...,...
9800,P03070,U13963,75.0,70.0
9801,P03070,U13964,70.0,75.0
9803,P03071,U13966,51.0,64.0
9804,P03071,U13967,50.0,51.0


In [94]:
train = train.sample(frac=1).reset_index(drop=True)

In [95]:
train[:int(len(train)*0.2)].to_csv('./pytorch/data/validation.csv', index=False)
train[int(len(train)*0.2):].to_csv('./pytorch/data/train.csv', index=False)

preprocessing test data

In [128]:
train = pd.read_csv('./train.csv')

In [65]:
# mean_array = np.zeros((1000, 12))

In [69]:
# for uid in train['UID']:
#     mean_array = mean_array + pd.read_csv(f'ecg/{uid}.csv').to_numpy()

In [72]:
# mean_array/=9806

In [77]:
# pd.DataFrame(mean_array.astype('int')).to_csv('ecg/mean_array.csv', index=False)

In [78]:
# mean_array = pd.DataFrame(mean_array.astype('int'))

In [ ]:
PID = ''
bdf = pd.DataFrame()
for index in range(len(train)):
    col = train.loc[index, :]
    uid = col['UID']
    if PID != col['PID']:
        PID = col['PID']
        bdf = pd.read_csv(f'ecg/{uid}.csv')
    else:
        df = pd.read_csv(f'ecg/{uid}.csv')
        pd.concat([df, bdf], axis=1).to_csv(f'./pytorch/data/ecg/{uid}.csv', index=False)
        bdf = df

In [ ]:
train.drop_duplicates(subset=['PID'], keep='last')

In [ ]:
test = pd.read_csv('./test.csv')

In [ ]:
test_uid = test.merge(train.drop_duplicates(subset=['PID'], keep='last')[['PID', 'UID']], on='PID', how='left')

In [ ]:
for uid_x, uid_y in zip(test_uid['UID_x'], test_uid['UID_y']):
    pd.concat([pd.read_csv(f'ecg/{uid_x}.csv'), pd.read_csv(f'ecg/{uid_y}.csv')], axis=1).to_csv(f'./pytorch/data/ecg/{uid_x}.csv', index=False)

In [ ]:
test = test.merge(train.drop_duplicates(subset=['PID'], keep='last')[['PID', 'EF']], on='PID', how='left')
test = test.rename({'EF': 'BEF'}, axis=1)

In [ ]:
test.to_csv('./pytorch/data/test.csv', index=False)

normalize data

In [44]:
std_dict = {'leadI':Normalizer(), 'leadII':Normalizer(), 'leadIII':Normalizer(),
            'leadaVR':Normalizer(), 'leadaVL':Normalizer(), 'leadaVF':Normalizer(),
            'leadV1':Normalizer(), 'leadV2':Normalizer(), 'leadV3':Normalizer(),
            'leadV4':Normalizer(), 'leadV5':Normalizer(), 'leadV6':Normalizer()}

In [45]:
file_list = os.listdir('ecg')

In [46]:
for file in file_list:
    df = pd.read_csv(f'ecg/{file}')
    for col in df.columns:
        std_dict[col].fit(df[col].values.reshape(1, -1))

In [61]:
for file in file_list:
    df = pd.read_csv(f'ecg/{file}')
    for col in df.columns:
        df[col] = std_dict[col].transform(df[col].values.reshape(1, -1)).reshape(-1)
    df.to_csv(f'./pytorch/data/ecg/{file}', index=False)